# Кастомные модели атрибуции
## Подготовка
### Установка необходимых программ

Для работы нам понадобится:
* СУБД ClickHouse
* Python 2.7 + несколько библиотек (pandas, requests, plot.ly)
* Jupyter Notebook

#### Установка ClickHouse
ClickHouse можно установить на Linux (подробно про установку написано в документации https://clickhouse.yandex/).

На Windows или Mac можно запустить ClickHouse под Docker'ом: первая команда поднимает clickhouse-server на порту 8123, а вторая - позволяет подключиться к консольному ClickHouse клиенту.
```
docker run -d --name clickhouse-server --publish=8123:8123 --publish=9000:9000 yandex/clickhouse-server
docker run -it --rm --link clickhouse-server:9000 yandex/clickhouse-client --host clickhouse-server
```

#### Установка python и jupyter notebook

Python можно скачать на [официальном сайте](https://www.python.org/getit/). 
Также в скрипте будут использованы некоторые дополнительные библиотеки, их проще всего поставить с помощью менеджера пакетов pip ([инструкция по установке pip](https://pip.pypa.io/en/stable/installing/))

```
pip install pandas jupyter requests plotly
```

Для того, чтобы запустить jupyter notebook, достаточно выполнить в консоли команду (после этого web-interface будет доступно по адресу http://localhost:8888):

```jupyter notebook```

### Выгрузка данных из LogsAPI в ClickHouse
Проще всего загрузить данные из LogsAPI в ClickHouse с помощью скрипта на GitHub'e.
Прежде всего нужно указать всю информацию в конфиге.

В конфиге собраны основные параметры и будет достаточно задать их один раз и затем пользоваться. Для данной задачи я использовала вот такой конфиг:

```
{
	"token" : "<insert>",
	"counter_id": "6375151",
	"visits_fields": [
	    "ym:s:counterID",
	    "ym:s:dateTime",
	    "ym:s:date",
	    "ym:s:visitDuration",
	    "ym:s:bounce",
	    "ym:s:pageViews",
	    "ym:s:goalsID",
	    "ym:s:firstPartyCookie",
	    "ym:s:lastDirectConditionType",
	    "ym:s:purchaseRevenue",
	    "ym:s:purchaseID",
	    "ym:s:lastTrafficSource",
	    "ym:s:lastAdvEngine",
	    "ym:s:lastSearchEngineRoot",
	    "ym:s:visitID",
	    "ym:s:UTMMedium",
	    "ym:s:startURL"
	],
	"log_level": "DEBUG",
	"retries": 1,
	"retries_delay": 60,
	"clickhouse": {
		"host": "http://localhost:8123",
		"user": "",
		"password": "",
		"visits_table": "visits_all",
		"hits_table": "hits_all",
		"database": "default"
	}
}
```

Часть параметров задается непосредственно при вызове функции в CLI options. Например, источник (хиты или визиты) и период выгрузки.
У скрипта есть несколько режимов работы. 
Во-первых, можно задать конкретные период (`start_date` и `end_date`):

```python metrica_logs_api.py -source visits -start_date 2015-12-25 -end_date 2015-12-25```

Другая опция это использовать один из режимов:
* __history__ - выгружает все данные с даты создания счетчика до позавчера 
* __regular__ - рекомендуемый режим для регулярных выгрузок, каждый день будет подтягивать данные за позавчера
* __regular_early__ - как __regular__ только берет вчерашний день (данные за вчера могут быть неполными, так что рекомендуется все-таки использовать __regular__)

```python metrica_logs_api.py -source visits -mode regular```

### Полезные функции

Напишем функции для интеграции с ClickHouse: первая функция просто возвращает результат из DataBase, вторая же преобразует его в pandas DataFrame.

In [2]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import StringIO

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

## Разминка или пара простых SQL запросов
Для начала посмотрим, какие данные есть в выгруженной с помощью Logs API таблице `visits_all`

In [3]:
q = '''DESCRIBE visits_all'''
print get_clickhouse_data(q)

CounterID	UInt32		
DateTime	DateTime		
Date	Date		
Duration	UInt32		
IsBounce	UInt8		
PageViews	Int32		
GoalsID	Array(UInt32)		
FirstPartyCookie	UInt64		
DirectConditionType	UInt8		
PurchaseRevenue	Array(Float64)		
PurchaseID	Array(String)		
LastSearchEngine	String		
LastAdvEngine	String		
LastTraficSource	String		
VisitID	UInt64		
UTMMedium	String		
StartURL	String		
ImpressionsProductID	Array(String)		



In [4]:
q = '''
    SELECT * FROM visits_all 
    LIMIT 5
    '''
print get_clickhouse_data(q)

6375151	2015-12-01 09:31:49	2015-12-01	4	1	1	[]	0	0	[]	[]	yandex	market	ad	5934905382539739018	cpc	http://220-volt.ru/catalog-156860/?ref=yamar&ymclid=48951480183663227117020	['156860']
6375151	2015-12-01 21:33:12	2015-12-01	0	1	1	[]	0	0	[]	[]	yandex		organic	5935082670770524045		http://m.220-volt.ru/catalog/svetodiodnyj-zanaves/	[]
6375151	2015-12-01 08:54:22	2015-12-01	1	1	1	[]	0	0	[]	[]	yandex	market	ad	5934896179407679609	cpc	http://220-volt.ru/catalog-210053/?ref=yamar&ymclid=48949242555794435398093	['210053']
6375151	2015-12-01 10:56:43	2015-12-01	0	1	1	[]	0	0	[]	[]			direct	5934926249142175607		http://ulyanovsk.220-volt.ru/catalog-59739/	[]
6375151	2015-12-01 15:25:47	2015-12-01	171	0	6	[]	0	0	[]	[]			direct	5934992373385930594		http://m.220-volt.ru/catalog/2-55/	[]



In [8]:
q = '''
    SELECT min(Date), max(Date)
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,min(Date),max(Date)
0,2015-12-01,2015-12-31


Посчитаем общее число визитов, пользователей, покупок, а также выручку.

__Disclaimer: выручка была умножена на случайное число.__

In [9]:
q = '''
    SELECT 
        count() as TotalVisits,
        uniq(FirstPartyCookie) as TotalUsers,
        sum(length(PurchaseID)) as TotalPurchases,
        sum(arraySum(PurchaseRevenue)) as TotalRevenue
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,TotalVisits,TotalUsers,TotalPurchases,TotalRevenue
0,5451184,2976586,70464,8336006970


Посмотрим на те же показатели в разбивке по пользователям.

In [10]:
q = '''
    SELECT 
        count() as TotalVisits,
        uniq(FirstPartyCookie) as TotalUsers,
        sum(length(PurchaseID)) as TotalPurchases,
        sum(arraySum(PurchaseRevenue)) as TotalRevenue,
        FirstPartyCookie
    FROM visits_all
    GROUP BY FirstPartyCookie
    ORDER BY TotalVisits DESC
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''
%time get_clickhouse_df(q)

CPU times: user 5.57 ms, sys: 2.87 ms, total: 8.44 ms
Wall time: 3.3 s


,TotalVisits,TotalUsers,TotalPurchases,TotalRevenue,FirstPartyCookie
0,435802,1,5179,654692984,0
1,235,1,0,0,1447747427476392199
2,164,1,1,14592,1447125777179508541
3,133,1,0,0,1446469321642514082
4,133,1,0,0,1449930383269506388


## Пути до покупки
Для начала выгрузим визиты, в которых были совершены покупки

In [11]:
q = '''
    SELECT 
        LastTraficSource,
        FirstPartyCookie,
        VisitID
    FROM visits_all
    WHERE (FirstPartyCookie != 0) AND (length(PurchaseID) != 0)
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,LastTraficSource,FirstPartyCookie,VisitID
0,direct,14466379051005714998,5939061898048872152
1,organic,1450082044461071361,5939344097664677658
2,referral,1449992588546282437,5939169643029919526
3,organic,1446633457145398431,5939217871051015961
4,direct,1448190872275658371,5939249704137096911


Для того, чтобы построить пути, т.е. для каждой покупки определить, с какими источниками пользователь появлялся на сайте ранее, нужно сделать JOIN таблицы с самой собой по идентификатору пользователя `FirstPartyCookie`.

In [13]:
q = '''
    SELECT 
        FirstPartyCookie,
        VisitID,
        PurchaseTime,
        DateTime,
        LastTraficSource
    FROM
        (SELECT 
            FirstPartyCookie,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (FirstPartyCookie != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            FirstPartyCookie,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING FirstPartyCookie
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 5.67 ms, sys: 1.89 ms, total: 7.56 ms
Wall time: 2.48 s


,FirstPartyCookie,VisitID,PurchaseTime,DateTime,LastTraficSource
0,14466379051005714998,5939061898048872152,2015-12-13 03:24:43,2015-12-04 08:29:26,direct
1,14466379051005714998,5939061898048872152,2015-12-13 03:24:43,2015-12-26 07:55:14,direct
2,14466379051005714998,5939061898048872152,2015-12-13 03:24:43,2015-12-22 18:55:30,direct
3,14466379051005714998,5939061898048872152,2015-12-13 03:24:43,2015-12-17 10:34:41,direct
4,14466379051005714998,5939061898048872152,2015-12-13 03:24:43,2015-12-13 03:24:43,direct


Объединим все по `VisitID` и с помощью функции `groupArray` соберем все источника трафика в массивы

In [14]:
q = '''
    SELECT 
        VisitID,
        groupArray(LastTraficSource) as Sources
    FROM
        (SELECT 
            FirstPartyCookie,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (FirstPartyCookie != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            FirstPartyCookie,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING FirstPartyCookie
    WHERE PurchaseTime >= DateTime
    GROUP BY VisitID
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 5.62 ms, sys: 2.09 ms, total: 7.71 ms
Wall time: 3.73 s


,VisitID,Sources
0,5942036913160910687,"['ad','ad','ad','ad','ad']"
1,5944887285864662889,"['ad','internal','ad']"
2,5936327790758690505,['ad']
3,5935674958795425569,['social']
4,5937596701532043135,"['organic','organic','referral','ad']"


И, в конце концов, посчитаем для каждого пути количество покупок и построим топ.

In [15]:
q = '''
    SELECT 
        Sources, 
        sum(NumPurchases) as TotalNumPurchases
    FROM
        (SELECT 
            any(NumPurchases) as NumPurchases,
            VisitID,
            groupArray(LastTraficSource) as Sources
        FROM
            (SELECT 
                length(PurchaseID) as NumPurchases,
                FirstPartyCookie,
                DateTime as PurchaseTime, 
                VisitID
            FROM visits_all
            WHERE (FirstPartyCookie != 0) AND (length(PurchaseID) != 0))
            ALL LEFT JOIN
            (SELECT 
                FirstPartyCookie,
                LastTraficSource,
                DateTime
            FROM visits_all
            ORDER BY DateTime)
            USING FirstPartyCookie
        WHERE PurchaseTime >= DateTime
        GROUP BY VisitID)
    GROUP BY Sources
    ORDER BY TotalNumPurchases DESC
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 5.43 ms, sys: 2.45 ms, total: 7.89 ms
Wall time: 2.34 s


,Sources,TotalNumPurchases
0,['organic'],13774
1,['ad'],7447
2,['direct'],4883
3,"['ad','ad']",2200
4,"['organic','organic']",2159


Получившиеся результаты уже достаточно интересны. Но, на самом деле, мы хотим разделять источники на более мелкие группы, а не только на те, которые уже есть в Метрике:
* __CPA__ (по условию `UTMMedium` равно `CPA` или `cpa`)
* __CPC__ (по условию `UTMMedium` равно `cpc`)
* __MarketPlace__ (по условию `UTMMedium` равно `marketplace` или это рекламный трафик с Яндекс.Маркета)
* __Ретаргетинг__ (определяем по `DirectConditionType` или по наличию параметра `ref=ga_ref` в URL)
* __Media__ - остальная реклама
* Хотим объединить Прямые заходы,Переходы с сохраненных страниц и Внутренние переходы в общую группу __Direct__
* Остальные источники оставим, как в Метрике

In [17]:
q = '''
    SELECT 
        LastTraficSource,
        UTMMedium,
        StartURL,
        LastAdvEngine, 
        LastSearchEngine,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''
%time get_clickhouse_df(q)

CPU times: user 7.27 ms, sys: 1.97 ms, total: 9.23 ms
Wall time: 139 ms


,LastTraficSource,UTMMedium,StartURL,LastAdvEngine,LastSearchEngine,Source,SourceDetailed
0,ad,cpc,http://220-volt.ru/catalog-156860/?ref=yamar&y...,market,yandex,cpc,cpc market
1,organic,NaN,http://m.220-volt.ru/catalog/svetodiodnyj-zana...,NaN,yandex,organic,organic yandex
2,ad,cpc,http://220-volt.ru/catalog-210053/?ref=yamar&y...,market,yandex,cpc,cpc market
3,direct,NaN,http://ulyanovsk.220-volt.ru/catalog-59739/,NaN,NaN,direct,direct
4,direct,NaN,http://m.220-volt.ru/catalog/2-55/,NaN,NaN,direct,direct
5,organic,NaN,http://omsk.220-volt.ru/catalog/,NaN,yandex,organic,organic yandex
6,ad,NaN,http://m.220-volt.ru/catalog-53213/?ref=ga_ret,google_adwords,NaN,media_ad,media_ad google_adwords
7,ad,NaN,http://m.220-volt.ru/catalog-220325/?ref=ga_ret,google_adwords,NaN,media_ad,media_ad google_adwords
8,organic,NaN,http://kemerovo.220-volt.ru/catalog/svarochie-...,NaN,yandex,organic,organic yandex
9,organic,NaN,http://220-volt.ru/catalog-136605/,NaN,google,organic,organic google


Добавляем в скрипт для расчета путей конверсии определение категорий и получаем пути по новым переопределенным каналам.

In [18]:
q = '''
SELECT 
    SourcesDetailed, 
    sum(VisitPurchases) as Purchases
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(FirstPartyCookie) as FirstPartyCookie,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases
        FROM
            (SELECT
                FirstPartyCookie,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    FirstPartyCookie,
                    DateTime as PurchaseTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all 
                WHERE FirstPartyCookie != 0 AND VisitPurchases != 0)
            ALL LEFT JOIN
                (SELECT
                    FirstPartyCookie,
                    DateTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all 
                 ORDER BY DateTime)
            USING FirstPartyCookie
            WHERE PurchaseTime >= DateTime)
        GROUP BY VisitID)
    GROUP BY SourcesDetailed
    ORDER BY Purchases DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 6.29 ms, sys: 2.48 ms, total: 8.77 ms
Wall time: 5.16 s


,SourcesDetailed,Purchases
0,['organic yandex'],8532
1,['direct'],4707
2,['organic google'],3989
3,['cpc market'],3690
4,['cpa other'],2249
5,['cpc yandex'],1966
6,['cpc other'],1531
7,"['organic yandex','organic yandex']",1321
8,"['direct','direct']",1164
9,['email'],1044


## Модели аттрибуции

### Модель по последнему переходу

In [19]:
q = '''
    SELECT 
        count() as Visits,
        sum(arraySum(PurchaseRevenue)) as Revenue,
        sum(length(PurchaseID)) as Purchases,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    WHERE FirstPartyCookie != 0
    GROUP BY Source, SourceDetailed
    FORMAT TabSeparatedWithNames
'''

%time last_raw_df = get_clickhouse_df(q).groupby(['Source', 'SourceDetailed']).sum()

CPU times: user 11.7 ms, sys: 5.75 ms, total: 17.4 ms
Wall time: 1.71 s


In [20]:
last_raw_df_norm = last_raw_df.apply(lambda x: 100.*x/last_raw_df.sum(), axis = 1)

### Кастомная модель с поведением

Напишем базовую функцию для расчета модели атрибуции:
* будем учитывать только источники без отказа
* источники, в которых была достигнута цель "Добавление в корзину" будет иметь вес на 50% обычного
* источники визитов, в которых были просмотры товаров будут иметь весь 10% больше обычного

In [36]:
def get_attribution_data_sample(i, N, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            groupArray(HasCart) as HasCarts,
            groupArray(HasImpression) as HasImpressions,
            VisitID,
            any(FirstPartyCookie) as FirstPartyCookie,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x, y -> 1 + 0.5*x + 0.1*y, HasCarts, HasImpressions) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                FirstPartyCookie,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                HasCart,
                HasImpression,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    FirstPartyCookie,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE FirstPartyCookie != 0)
            ALL LEFT JOIN
                (SELECT
                    FirstPartyCookie,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed,
                    has(GoalsID, 552829) as HasCart,
                    length(ImpressionsProductID) > 0 as HasImpression
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition}
                 )
            USING FirstPartyCookie
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data(where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_sample(i, 10, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [37]:
%time not_bounce_df = get_attribution_data(where_condition = 'WHERE IsBounce = 0')

CPU times: user 61.8 ms, sys: 7.54 ms, total: 69.4 ms
Wall time: 26.8 s


In [38]:
not_bounce_df_norm = not_bounce_df.apply(lambda x: 100.*x/not_bounce_df.sum(), axis = 1)

### Кастомная модель c перевзвешенными каналами и затуханием по времени

In [24]:
traffic_sources_coeffs = {
    'direct': 0.1,
    'referral': 0.2,
    'social': 0.3,
    'cpa': 0.5,
    'cpc': 0.5,
    'retargeting': 0.3,
    'organic': 0.4,
    'email': 0.4,
    'media_ad': 0.5,
    'marketplace': 0.5
}

In [25]:
def get_attribution_data_pos_sample(i, N, traffic_sources_coeffs, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(FirstPartyCookie) as FirstPartyCookie,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x -> transform(x, [{traffic_sources}], [{traffic_sources_coeffs}], 0), Sources) as SourcesRawCoefs,
            arrayMap(x -> 1/(length(SourcesRawCoefs) - x), range(length(SourcesRawCoefs))) as PositionCoefs,
            arrayMap(x, y -> x * y, SourcesRawCoefs, PositionCoefs) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                FirstPartyCookie,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    FirstPartyCookie,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE FirstPartyCookie != 0)
            ALL LEFT JOIN
                (SELECT
                    FirstPartyCookie,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition})
            USING FirstPartyCookie
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        traffic_sources = ', '.join(map(lambda x: "'%s'" % x, traffic_sources_coeffs.keys())),
        traffic_sources_coeffs = ', '.join(map(str, traffic_sources_coeffs.values())),
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data_pos(traffic_sources_coeffs, where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_pos_sample(i, 10, traffic_sources_coeffs, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [26]:
%time not_bounce_pos_df = get_attribution_data_pos(traffic_sources_coeffs, where_condition = 'WHERE IsBounce = 0')

CPU times: user 66.7 ms, sys: 8.93 ms, total: 75.6 ms
Wall time: 23.9 s


In [27]:
not_bounce_pos_df_norm = not_bounce_pos_df.apply(lambda x: 100.*x/not_bounce_pos_df.sum(), axis = 1)

## Сравним получившиеся модели
Для визуализации будем использовать библиотеку `plot.ly`, которая позволяет строить интерактивные визуализации прямо в Jupyter Notebook'e и к тому же она open source.

In [28]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def show_bar_plot(cmp_df):
    data = []
    for column in cmp_df.columns:
        trace = go.Bar(
            x = cmp_df.index.values,
            y = cmp_df[column].values,
            name = column
        )
        data.append(trace)
    layout = go.Layout({'xaxis': {'tickangle': 45}})
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, show_link=False)

1.12.9


In [29]:
dct = {
    'last click': last_raw_df_norm,
    'behaviour': not_bounce_df_norm,
    'time decay & weights': not_bounce_pos_df_norm,
}

In [30]:
def get_comparison(parameter, detalization, show_delta = False):
    cmp_df = pd.DataFrame()
    for item in dct:
        cmp_df[item] = dct[item].reset_index().groupby(detalization)[parameter].sum()
    cmp_df = cmp_df.sort_values('last click', ascending = False)
    if not show_delta:
        return cmp_df
    cmp_df_norm = cmp_df.apply(lambda x: x - cmp_df['last click']).drop('last click', axis = 1).sort_values('behaviour')
    return cmp_df_norm

In [35]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=False))

In [32]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=True))

In [33]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed'))

In [34]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed', show_delta=True))